In [18]:
using Flows, PeriodicShadowing, PyPlot; pygui(true)

true

In [2]:
include("lorenz.jl");

In [124]:
# Problem Parameters - the parameter ρ defaults to 28
Δt = 0.5e-2;

In [125]:
# nonlinear right hand side and associated integrator
f = LorNLinEq()
ϕ = flow(f, 
         RK4(zeros(3), :NORMAL), 
         TimeStepConstant(Δt));

In [126]:
# linear homogeneous integrator, obtained by 
# coupling the linear and nonlinear equations.
l_h = LorLinEq()
ψ   = flow(couple(f, l_h), 
           RK4(couple(zeros(3), zeros(3)), :NORMAL), 
           TimeStepConstant(Δt));

In [127]:
# linear inhomogeneous integrator, obtained by 
# coupling the linear and nonlinear equations
l_nh = LorLinEq(dfdρ_forcing)
ρ    = flow(couple(f, l_nh), 
            RK4(couple(zeros(3), zeros(3)), :NORMAL), 
            TimeStepConstant(Δt));

In [181]:
# Parameters of the shooting algorithm

# initial condition
x0 = ϕ(Float64[1, 1, 1], (0, 100)) 

# time span
T = 250

# number of shooting intervals (it is best if this is selected such 
# that the nonlinear solver hits the endpoints of the shooting interval
# exactly)
N = 50;

In [182]:
# Get shooting points
x0s = get_shooting_points(ϕ, x0, T, N);

In [183]:
# Now solve the problem. This will return the solution at the shooting points and the 
# period gradient
y0s, TpT = solve_ps_plc_tan(ψ, ρ, ϕ, f, copy(x0s), T);

In [184]:
# Let us now plot the solution. We first need to define a non-homogeneous linear 
# integrator to integrate the full equations. We do this by passing two forcings
# that will be added to the normal homogeneous equations. 
l_nh_χ = LorLinEq(dfdρ_forcing, f_forcing(TpT))
ρ_χ    = flow(couple(f, l_nh_χ), 
            RK4(couple(zeros(3), zeros(3)), :NORMAL), 
            TimeStepConstant(Δt));

In [185]:
# We need to define a monitor object, to save the solution. When we integrate the 
# coupled equations we will pass to the monitor objects of type `Coupled`, which
# include the state and the perturbation. We get the perturbation with the function 
# `last` (as opposed to first for the state part), and the save the third component
mon = Monitor(couple(zeros(3), zeros(3)), xy->last(xy)[3]);

In [186]:
# Now we propagate the non-homogeneous propagator coupled to the nonlinear
# equations. Note how we couple the correct state and perturbation components
# and how we make a copy, to avoid corrupting the data. I am also using the
# function ith_span, to integrate the equations over the correct time span.
# this will be usefult later for plotting.

# for each interval, we save the history of z and the corresponding time here
z_all = Float64[]
t_all = Float64[]

for i = 1:N
    # propagate. Note that we need to reset the monitor at each step
    ρ_χ(couple(copy(x0s[i]), copy(y0s[i])), ith_span(i, N, T), reset!(mon))
    
    # save
    append!(z_all, samples(mon))
    append!(t_all, times(mon))
end  

In [187]:
# Let's now plot the solution!
figure(1)
clf()

plot(t_all, z_all)

# we also add lines, to denote the shooting intervals
for i = 0:N-1
    axvline(i*T/N, color="0.3", zorder=0)
end

# add labels
xlabel(L"t")
ylabel(L"z^\prime(t)")

PyObject <matplotlib.text.Text object at 0x104f77710>

In [188]:
# Now we need to compute certain integrals, namely the sensitivity integral.
# To do so, we define a quadrature function first. Note the signature:
# this function will receive a nested coupled object, containing the state 
# the perturbation and the current value of the quadrature, which is shown
# by how I first unpack the term xyq. 
function quadfun(t, xyq::Coupled{<:Coupled, Vector{Float64}}, dqdt)
    x = first(first(xyq))
    y =  last(first(xyq))
    q =        last(xyq)
    dqdt[1] = y[3]
    return dqdt
end

# now we define a special integrator, defined by the coupled nonlinear/linearised
# equations, (forget about the second argument, it used for implicit-explicit
# integrators such as CB3R2R3e), plus the quadrature equation. Note that we do
# not have to define the type for RK4 as a nested coupled type. However, since
# we only integrate one quadrature equation, we define zeros(1) for that one.
l_nh_χ = LorLinEq(dfdρ_forcing, f_forcing(TpT))
ρ_χ_quad = flow(couple(f, l_nh_χ), nothing, quadfun,
            RK4(couple(zeros(3), zeros(3)), zeros(1), :NORMAL), 
            TimeStepConstant(Δt));

In [189]:
# now we can compute the gradient of the average! This is quite 
# close to the value obtained from finite-differences! GOOD
quadrature_(ρ_χ_quad, y0s, x0s, Float64[0.0], T)

1-element Array{Float64,1}:
 1.01559